In [20]:
import pandas as pd
from pandas import DataFrame
import numpy as np

df = pd.read_csv('data/spam.csv', encoding = 'latin1')

In [2]:
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [11]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/oscar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
df_listed = DataFrame(columns=('id', 'stf', 'text'))
df_listed.append([1, 'ham', 'bottom text'])
print(df_listed['stf'])
'''
import re

for text in df['v2']:
    df_listed['v2'] = re.sub("[^\w]", " ",  text).split()
    
for'''

'''
for text in df['v2']:
    text.apply(lambda x: [item for item in x if item not in stop])
    print(text)
'''

Series([], Name: stf, dtype: object)


"\nfor text in df['v2']:\n    text.apply(lambda x: [item for item in x if item not in stop])\n    print(text)\n"